In [1]:
function calculate_columns(node)
    node_pool=Array{Array{Array{Int,1},1},1}(undef,D.m)

    for m in 1:D.m
        node_pool[m]=Array{Array{Int,1},1}()
        push!(node_pool[m],vcat(10000,ones(Int,D.n+1)))
        for k in 2:size(column_pool[m],1)
            add=true
            
            # TO COMPLETE
            # There are two conditions under which the column should be eliminated:
            #  1. If the branching constraint x_ji<=0 apply and job i is assigned to machine j
            for (machine,job) in tree[node].setzero
                # TO COMPLETE
            end
            #  2. If the branching constraint x_ji>=1 apply and job i is not assigned to machine j
            for (machine,job) in tree[node].setone
                # TO COMPLETE
            end
            # TO COMPLETE

            if add push!(node_pool[m],column_pool[m][k]) end
        end
    end

    return node_pool
end

calculate_columns (generic function with 1 method)

In [2]:
# Translate the α solution to x 
function calculate_xsol(mastersol,node_pool)
    x=Array{Array{Float32},1}(undef,D.m)
    for m in 1:D.m
        x[m]=Array{Float32,1}(undef,D.n)
    end

    for m in 1:D.m
        for j in 1:D.n
            for k in 1:size(node_pool[m],1))
                if mastersol[m][k] == 1
                    x[m,j] = node_pool[m][k]
                end
            end
        end
    end
    return x
end

calculate_xsol (generic function with 1 method)

In [3]:
function Process_Node(nodeindex)
    println("\e[93m Processing node $nodeindex \e[00m")
    println("Variables set to zero are:", tree[nodeindex].setzero)
    println("Variables set to one are:", tree[nodeindex].setone)
    global nodestobedeleted = []
    
    # Calculate columns that are compatible with the branching constraints
    # Function calculate_columns TO COMPLETE (see above)
    node_pool = calculate_columns(nodeindex)
    
    # Recover the initial restricted master LP:
    # constraint and variable references as well as the model itself
    job,machine,alpha,mastermodel = node_master(node_pool)
    
    while true
        # Solve the restricted master LP
        optimize!(mastermodel)
        
        node_infeasible = false
        
        # Recover the optimal dual solution for the first set of constraints
        π = JuMP.dual.(job)
        
        # Recover the optimal dual solution for the second set of constraints
        σ = JuMP.dual.(machine)
        
        # Recover the current master objective value
        value = JuMP.objective_value(mastermodel)
        
        # Extract the solution
        solution=Array{Array{Float32,1},1}(undef,D.m)
        for m in 1:D.m
            solution[m]=Array{Float32,1}(undef,size(node_pool[m],1))
            for k in 1:size(node_pool[m],1)
                solution[m][k]=JuMP.value.(alpha[m])[k]
            end
        end
        solflat = collect(Iterators.flatten(solution))

        # Check whether the solution is integer
        # If so update the bounds
        if maximum(solflat-floor.(solflat)) <= ϵ
            # Found a feasible solution update the best known upper bound
            if value<=UB global UB=value end
            println("\e[32m Feasible solution with value $value found \e[00m")
            for i in 1:length(Queue)-1
                if tree[Queue[i]].lb >= UB
                    push!(nodestobedeleted,i)
                end
            end
        end
        
        # Update the node lower and upper bounds
        nodeub = UB# TO COMPLETE
        nodelb = value # value # TO COMPLETE
        minobj = 0
        for i in 1:D.m
            # Solve the machine subproblem. 
            # Recover the objective value and the optimal pattern solution
            SPobj,column = machine_subproblem(i,π,nodeindex)
            
            if SPobj<Inf 
                # Extract the pattern from the solution
                pattern=findall(x->x!=0,column)
                # Calculate the reduced cost of the pattern
                reduced_cost = SPobj - sum(σ[m] for m in D.m) # TO COMPLETE
                
                if reduced_cost<=minobj minobj=reduced_cost end
                
                # If negative reduced cost pattern is found add it to the master
                if reduced_cost<0
                    # Calculate the column's objective function contribution
                    column_cost = sum(D.c[i,j]*column[j] for j in 1:D.n) # TO COMPLETE
                    
                    
                    # Store column in the column pool
                    push!(column_pool[i],vcat(column_cost,vcat(column,1)))
                    # Store column in the node pool
                    push!(node_pool[i],vcat(column_cost,vcat(column,1)))
                    # ADD COLUMN TO MODEL
                    # Add the new variable to the vector of variables for the machine
                    push!(alpha[i],@variable(mastermodel,lower_bound=0))
                    # Set the name of the new variable
                    set_name(alpha[i][end], "alpha_$i,$(size(column_pool[i],1))")
                    # Set the coefficient of the variable in constraints job
                    for j in 1:D.n
                        set_normalized_coefficient(job[j], alpha[i][end], column[j])
                    end
                    # Set the coefficient of the variable in constraints machine
                    set_normalized_coefficient(machine[i], alpha[i][end], 1) 
                    
                    # Update the objective function    
                    set_objective_function(mastermodel, objective_function(mastermodel) + column_cost*alpha[i]) #TO COMPLETE
                    # ADD COLUMN TO MODEL
                    println("\e[34m Pattern with items $pattern added for machine $i \e[00m")
                end
                # Complete the lower bound calculation
                nodelb += SPobj
            else
                node_infeasible = true
                println("\e[31m Node is infeasible \e[00m")
                break
            end
        end

        # If there is a subproblem that is infeasible then the node is infeasible RETURN EMPTY
        if node_infeasible return [] end

        # If lower bound is improved UPDATE
        if nodelb>=tree[nodeindex].lb tree[nodeindex].lb=nodelb end

        # If converged STOP
        if 2*abs((nodeub-tree[nodeindex].lb))/abs((nodeub+tree[nodeindex].lb))<ϵ || minobj>=0
            println("Node relaxation is solved to optimality")
            println("Node upper bound is $nodeub")
            println("Node lower bound is $(tree[nodeindex].lb)")
            # If the artificial column is used in the final solution
            # then the master LP is infeasible
            for m in 1:D.m
                if solution[m][1]>=ϵ
                    node_infeasible = true
                    println("\e[31m Node is infeasible \e[00m")
                end
            end
            # If infeasible RETURN EMPTY
            if node_infeasible return []
            # If not infeasible RETURN SOLUTION   
            # Function calculate_xsol which translates the solution in terms of
            # α variables to a solution in terms of x variables: TO COMPLETE (See above)    
            else return calculate_xsol(solution,node_pool) end
        end
    end
end

LoadError: UndefVarError: @variable not defined